In [205]:
import pandas as pd
import numpy as np

In [206]:
bess_tags = pd.read_csv('/Users/user/Documents/CBW/Data/final/CBW_Bess_tags_final2.csv', encoding='ISO-8859-1')

In [207]:
textdata_sent = pd.read_csv('/Users/user/Documents/CBW/Data/final/text_data_sent.csv', encoding='ISO-8859-1')

In [208]:
bess_tags[['biographyID','collectionID']].drop_duplicates().shape

(306, 2)

In [209]:
bess_tags.head()

,Content,Event,Type,para no,URI,author,biographyID,collectionID,personaName,title
0,after,name,stageOfLife,1.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...
1,culmination,name,stageOfLife,1.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...
2,middle,name,stageOfLife,2.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...
3,middle,name,stageOfLife,3.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...
4,middle,name,stageOfLife,4.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...


In [210]:
#bess_tags.loc[bess_tags['para no'].isnull(), 'para no'] = 0

bess_tags = bess_tags[~bess_tags['para no'].isnull()]

In [211]:
bess_tags['para no']=bess_tags['para no'].astype(int)

In [212]:
textdata_sent['para_key']= textdata_sent.CollectionID.apply(str) + '_' + textdata_sent.BiographyID.apply(str)+ '_' + textdata_sent.ParagraphNo.apply(str) 
bess_tags['para_key']= bess_tags.collectionID.apply(str) + '_' + bess_tags.biographyID.apply(str)+ '_' + bess_tags['para no'].apply(str) 

In [213]:
groupbykey =textdata_sent.groupby('para_key')['score'].mean()
groupbykey = pd.DataFrame(groupbykey)

In [214]:
groupbykey.head()

,score
para_key,
a001_bio04_1,0.290669
a001_bio04_2,0.000000
a001_bio04_3,0.376424
a001_bio04_4,-0.077556
a001_bio04_5,0.452558


In [215]:
bess_new = bess_tags[['para_key','author', 'personaName', 'para no','biographyID','collectionID']].drop_duplicates()

In [216]:
avg_sentiment = pd.merge(groupbykey, bess_new,on='para_key', how='left')

In [217]:
avg_sentiment.head()

,para_key,score,author,personaName,para no,biographyID,collectionID
0,a001_bio04_1,0.290669,Willis John Abbot,Cleopatra,1.0,bio04,a001
1,a001_bio04_2,0.000000,Willis John Abbot,Cleopatra,2.0,bio04,a001
2,a001_bio04_3,0.376424,Willis John Abbot,Cleopatra,3.0,bio04,a001
3,a001_bio04_4,-0.077556,Willis John Abbot,Cleopatra,4.0,bio04,a001
4,a001_bio04_5,0.452558,Willis John Abbot,Cleopatra,5.0,bio04,a001


In [218]:
total_paras = avg_sentiment.groupby(['collectionID','biographyID'],as_index=False)['para no'].max()

In [219]:
avg_sentiment = pd.merge(avg_sentiment,total_paras,how = 'left', on = ['biographyID','collectionID'])

In [220]:
avg_sentiment.head()

,para_key,score,author,personaName,para no_x,biographyID,collectionID,para no_y
0,a001_bio04_1,0.290669,Willis John Abbot,Cleopatra,1.0,bio04,a001,9.0
1,a001_bio04_2,0.000000,Willis John Abbot,Cleopatra,2.0,bio04,a001,9.0
2,a001_bio04_3,0.376424,Willis John Abbot,Cleopatra,3.0,bio04,a001,9.0
3,a001_bio04_4,-0.077556,Willis John Abbot,Cleopatra,4.0,bio04,a001,9.0
4,a001_bio04_5,0.452558,Willis John Abbot,Cleopatra,5.0,bio04,a001,9.0


In [221]:
avg_sentiment['time']= avg_sentiment['para no_x']/avg_sentiment['para no_y']
avg_sentiment['score']=avg_sentiment.score

In [222]:
#subsetting based on persona for plots

In [223]:
Queen = avg_sentiment.loc[avg_sentiment['personaName']=='Queen Victoria']

In [224]:
#list(lola.para_key.unique())

In [225]:
import plotly
plotly.tools.set_credentials_file(username='sjawarani', api_key='ZlyoNIh0X7NAU0znNqgE')
from plotly.plotly import iplot

In [226]:
#lola.para_key.strhead()

In [227]:
Queen = Queen.sort_values(['time'])

In [229]:
Queen = Queen.drop_duplicates()

In [230]:
Queen.para_key.unique()[0:10]

array(['a143_bio01_1', 'a143_bio01_2', 'a185_bio01_1', 'a143_bio01_3',
       'a270_bio01_1', 'a186_bio01_1', 'a143_bio01_4', 'a185_bio01_2',
       'a143_bio01_5', 'a143_bio01_6'], dtype=object)

In [202]:
import plotly.plotly as py
import plotly.graph_objs as go

In [231]:

trace0 = go.Scatter(
        x = Queen.time[Queen.para_key.str.contains("a664_bio12")] ,
        y = Queen.score[Queen.para_key.str.contains("a664_bio12")] ,
        mode='markers',
        marker = dict(
            size = 10,
            color = 'rgba(152, 0, 0, .8)',
            line = dict(
                width = 2,
                color = 'rgb(0, 0, 0)'
            )
        )
    )

    # trace1 = go.Scatter(
    #     x = lola.time[lola.para_key.str.contains('a095A_bio05')],
    #     y = lola.score[lola.para_key.str.contains('a095A_bio05')] ,
    #     marker = dict(
    #         size = 10,
    #         color = 'rgba(255, 182, 193, .9)',
    #         line = dict(
    #             width = 2,
    #         )
    #     )
    # )

data = [trace0]

layout = dict(title = 'Styled Scatter',
                  yaxis = dict(zeroline = False),
                  xaxis = dict(zeroline = False)
                 )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-scatter')


High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~sjawarani/0 or inside your plot.ly account where it is named 'styled-scatter'


In [232]:
Dora = avg_sentiment.loc[avg_sentiment['personaName']=='Sister Dora']
Dora = Dora.sort_values(['time'])
Dora = Dora.drop_duplicates()

In [234]:

trace0 = go.Scatter(
        x = Dora.time[Dora.para_key.str.contains("a730_bio05")] ,
        y = Dora.score[Dora.para_key.str.contains("a730_bio05")] ,
        mode='markers',
        marker = dict(
            size = 10,
            color = 'rgba(152, 0, 0, .8)',
            line = dict(
                width = 2,
                color = 'rgb(0, 0, 0)'
            )
        )
    )

    # trace1 = go.Scatter(
    #     x = lola.time[lola.para_key.str.contains('a095A_bio05')],
    #     y = lola.score[lola.para_key.str.contains('a095A_bio05')] ,
    #     marker = dict(
    #         size = 10,
    #         color = 'rgba(255, 182, 193, .9)',
    #         line = dict(
    #             width = 2,
    #         )
    #     )
    # )

data = [trace0]

layout = dict(title = 'Styled Scatter',
                  yaxis = dict(zeroline = False),
                  xaxis = dict(zeroline = False)
                 )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-scatter')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~sjawarani/0 or inside your plot.ly account where it is named 'styled-scatter'


In [236]:

trace0 = go.Scatter(
        x = Dora.time[Dora.para_key.str.contains("a207_bio11")] ,
        y = Dora.score[Dora.para_key.str.contains("a207_bio11")] ,
        mode='markers',
        marker = dict(
            size = 10,
            color = 'rgba(152, 0, 0, .8)',
            line = dict(
                width = 2,
                color = 'rgb(0, 0, 0)'
            )
        )
    )

    # trace1 = go.Scatter(
    #     x = lola.time[lola.para_key.str.contains('a095A_bio05')],
    #     y = lola.score[lola.para_key.str.contains('a095A_bio05')] ,
    #     marker = dict(
    #         size = 10,
    #         color = 'rgba(255, 182, 193, .9)',
    #         line = dict(
    #             width = 2,
    #         )
    #     )
    # )

data = [trace0]

layout = dict(title = 'Styled Scatter',
                  yaxis = dict(zeroline = False),
                  xaxis = dict(zeroline = False)
                 )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-scatter')



High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~sjawarani/0 or inside your plot.ly account where it is named 'styled-scatter'
